# Build A Distributed ORM From Scratch

    First of all we need to build a database connection driver

## Database Driver

In [66]:
# %load database/drivers.py
import threading
import sqlite3
import psycopg2
from pymongo import MongoClient
import pandas as pd
from database.models import Model
from utils.serializers import ModelSerializer

class Sqlite(threading.local):
    def __init__(self, database):
        super(Sqlite, self).__init__()
        self.database = database
        self.conn = sqlite3.connect(self.database, detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES)

        self.__tables__ = {}
        setattr(self, 'Model', Model)
        setattr(self.Model, '__db__', self)

    def create_table(self, model):
        tablename = model.__tablename__
        create_sql = ', '.join(field.create_sql() for field in model.__fields__.values())
        self.execute('create table {0} ({1});'.format(tablename, create_sql), commit=True)

        if tablename not in self.__tables__.keys():
            self.__tables__[tablename] = model

        for field in model.__refed_fields__.values():
            if isinstance(field, ManyToMany):
                field.create_m2m_table()

    def drop_table(self, model):
        tablename = model.__tablename__
        self.execute('drop table {0};'.format(tablename), commit=True)
        del self.__tables__[tablename]

        for name, field in model.__refed_fields__.items():
            if isinstance(field, ManyToMany):
                field.drop_m2m_table()

    def commit(self):
        self.conn.commit()

    def rollback(self):
        self.conn.rollback()

    def close(self):
        self.conn.close()

    def __enter__(self):
        return self

    def __exit__(self, *args):
        self.close()

    def execute(self, sql, commit=False):
            cursor = self.conn.cursor()
            #print(sql)
            cursor.execute(sql)
            if commit:
                self.commit()
            return cursor

class MongoDB(threading.local):
    """Not done yet"""
    def __init__(self, database):
        super(MongoDB, self).__init__()
        self.database = MongoClient()[database] # create mongo database
        self.conn = self.database

        self.__tables__ = {}
        setattr(self, 'Model', Model)
        setattr(self.Model, '__db__', self)    
    
    def create_table(self, model):
        tablename = model.__tablename__
        self.database[tablename] # create collection

        # register the given model as a table to database  if  not 
        if tablename not in self.__tables__.keys():
            self.__tables__[tablename] = model
 
        """ # still do not know what I am supposed to do with relations
        for field in model.__refed_fields__.values():
            if isinstance(field, ManyToMany):
                field.create_m2m_table()
        """
        return tablename
                   
    def drop_table(self, model):
        tablename = model.__tablename__
        # test
        self.database[tablename].drop()
        del self.__tables__[tablename]


    def commit(self):
        # what to do?
        #self.conn.commit()
        pass

    def rollback(self):
        # what to do?
        #self.conn.rollback()
        pass

    def close(self):
        # what
        #self.conn.close()
        pass

    def __enter__(self):
        return self

    def __exit__(self, *args):
        # self.close()
        print("exiting now")

    def execute(self, bson, commit=False):
            #print(sql)
            cursor.execute(sql)
            
            return cursor

class Cursor(object):
    
    def __init__(self, conn, query, step=20, forward=0):
        self._conn = conn
        self._query = query
        self._results = []

        self._step = step
        self._forward = forward

    async def execute(self):
        
        cursor = self._conn.execute(self._query)

        if self._forward:
            cursor.forward(self._forward)

        if not cursor:
            raise StopAsyncIteration()
        return cursor

    def __aiter__(self):
        return self

    async def __anext__(self):
        if self._cursor is None:
            self._results = await self.execute()

        if not self._results:
            self._forward = self._forward + self._step
            self._results = await self.execute()

        return self._results.pop(0)

class AsyncSqlite(Sqlite):
    def __init__(self, *args, **kwargs):
        super(AsyncSqlite, self).__init__(*args, **kwargs)

    async def execute(self, sql, commit=False):
        cursor = await Cursor(self.conn, sql).execute()
        if commit:
            self.commit()
        return cursor



In [68]:
# %load database/models.py

from database.queries import *
from utils.serializers import jsonify


"""
Create Field based classes here to represent the database columns
"""

class Field(object):
    """Base field object"""
    def __init__(self, column_type):
        self.column_type = column_type
        self.name = None

    def create_sql(self):
        """Return sql statement for create table."""
        return '"{0}" {1}'.format(self.name, self.column_type)


class Integer(Field):
    """SQLite Integer field"""
    def __init__(self):
        super(Integer, self).__init__('INTEGER')

    def sql_format(self, data):
        """sql query format of data"""
        return str(int(data))
    
    def _serialize_data(self, data):
        return str(int(data))


class Char(Field):
    """SQLite Char field"""
    def __init__(self, max_length=255):
        self.max_length = max_length
        super(Char, self).__init__('CHAR')

    def create_sql(self):
        return '"{0}" {1}({2})'.format(self.name, self.column_type, self.max_length)

    def sql_format(self, data):
        """sql query format of data"""
        return '"{0}"'.format(str(data))
        
    def _serialize_data(self, data):
        return str(data)

class Varchar(Field):
    """SQLite Varchar field"""
    def __init__(self, max_length=255):
        self.max_length = max_length
        super(Varchar, self).__init__('VARCHAR')

    def create_sql(self):
        return '"{0}" {1}({2})'.format(self.name, self.column_type, self.max_length)

    def sql_format(self, data):
        """sql query format of data"""
        return '"{0}"'.format(str(data))

class Text(Field):
    """SQLite Text field"""
    def __init__(self):
        super(Text, self).__init__('TEXT')

    def sql_format(self, data):
        """sql query format of data"""
        return '"{0}"'.format(str(data))

class DateTime(Field):
    def __init__(self):
        super(DateTime, self).__init__('DATETIME')

    def sql_format(self, data):
        return '"{0}"'.format(data.strftime('%Y-%m-%d %H:%M:%S'))
    
    def _serialize_data(self, data):
        
        return str(data)    

class PrimaryKey(Integer):
    def __init__(self):
        super(PrimaryKey, self).__init__()

    def create_sql(self):
        return '"{0}" {1} NOT NULL PRIMARY KEY'.format(self.name, self.column_type)


class ForeignKey(Integer):
    def __init__(self, to_table):
        self.to_table = to_table
        super(ForeignKey, self).__init__()

    def create_sql(self):
        return '{column_name} {column_type} NOT NULL REFERENCES "{tablename}" ("{to_column}")'.format(
            column_name=self.name,
            column_type=self.column_type,
            tablename=self.to_table,
            to_column='id'
        )


class ForeignKeyReverse(object):
    def __init__(self, from_table):
        self.from_table = from_table
        self.name = None
        self.tablename = None
        self.instance_id = None
        self.db = None
        self.from_model = None
        self.relate_column = None

    def update_attr(self, name, tablename, db):
        self.name = name
        self.tablename = tablename
        self.db = db
        self.from_model = self.db.__tables__[self.from_table]
        for k, v in self.from_model.__dict__.items():
            if isinstance(v, ForeignKey) and v.to_table == self.tablename:
                self.relate_column = k

    def all(self):
        return self._query_sql().all()

    def count(self):
        return self._query_sql().count()

    def _query_sql(self):
        return self.from_model.select().where(**{self.relate_column: self.instance_id})


class ManyToManyBase(object):
    def __init__(self, to_model):
        self.to_model = to_model

        self.name = None
        self.tablename = None
        self.db = None

        self.instance_id = None

        self.relate_model = None
        self.relate_table = None
        self.relate_column = None
        self.to_table = None
        self.to_column = None

    def update_attr(self, name, tablename, db):
        self.name = name
        self.tablename = tablename
        self.db = db

    def add(self, to_instance):
        insert = {
            self.relate_column: self.instance_id,
            self.to_column: to_instance.id
        }
        self.relate_model(**insert).save()

    def remove(self, to_instance):
        self.relate_model.delete(**{self.to_column: to_instance.id}).commit()

    def all(self):
        return self._query_sql().all()

    def count(self):
        return self._query_sql().count()

    def _query_sql(self):
        self.to_model = self.db.__tables__[self.to_table]

        relate_instances = self.relate_model.select().where(**{self.relate_column: self.instance_id}).all()
        to_ids = [str(getattr(instance, self.to_column)) for instance in relate_instances]
        where_sql = 'id in ({0})'.format(', '.join(to_ids))

        return self.to_model.select().where(where_sql)


class ManyToMany(ManyToManyBase):
    def __init__(self, to_model):
        super(ManyToMany, self).__init__(to_model)

    def update_attr(self, name, tablename, db):
        super(ManyToMany, self).update_attr(name, tablename, db)
        if self.to_model not in self.db.__tables__.values():
            raise DatabaseException('Related table "{0}" not exists'.format(self.to_model.__tablename__))

        self.to_table = self.to_model.__tablename__
        self.to_column = '{0}_id'.format(self.to_table)
        self.relate_column = '{0}_id'.format(self.tablename)

        class_name = '{0}_{1}'.format(self.to_table, self.tablename)
        class_attrs = {
            self.relate_column: ForeignKey(self.tablename),
            self.to_column: ForeignKey(self.to_table)
        }
        m2m_model = type(class_name, (Model, ), class_attrs)

        self.relate_model = m2m_model
        self.relate_table = getattr(m2m_model, '__tablename__')
        self.db.__tables__[self.relate_table] = m2m_model
        self.create_reversed_field()

    def create_m2m_table(self):
        self.db.create_table(self.relate_model)
        self.create_reversed_field()

    def drop_m2m_table(self):
        try:
            table_model = self.db.__tables__[self.relate_table]
        except KeyError:
            raise DatabaseException('Can not drop this table: "{0}" not exists'.format(self.relate_table))
        self.db.drop_table(table_model)
        self.delete_reversed_field()

    def create_reversed_field(self):
        field = ManyToManyBase(self.db.__tables__[self.tablename])
        field.db = self.db
        field.name = '{0}s'.format(self.tablename)
        field.to_table, field.tablename = self.tablename, self.to_table
        field.to_column, field.relate_column = self.relate_column, self.to_column
        field.relate_model, field.relate_table = self.relate_model, self.relate_table

        setattr(self.to_model, field.name, field)
        self.to_model.__refed_fields__[field.name] = field

    def delete_reversed_field(self):
        to_column = '{0}s'.format(self.tablename)
        delattr(self.to_model, to_column)
        del self.to_model.__refed_fields__[to_column]

class MetaModel(type):
    """
        Metamodel that initializes the database table model as creation of model class
    """
    def __new__(mcs, name, bases, attrs):
        if name == 'Model':
            return super(MetaModel, mcs).__new__(mcs, name, bases, attrs)

        cls = super(MetaModel, mcs).__new__(mcs, name, bases, attrs)

        if 'Meta' not in attrs.keys() or not hasattr(attrs['Meta'], 'db_table'):
            setattr(cls, '__tablename__', name.lower())
        else:
            setattr(cls, '__tablename__', attrs['Meta'].db_table)

        if hasattr(cls, '__db__'):
            getattr(cls, '__db__').__tables__[cls.__tablename__] = cls

        fields = {}
        refed_fields = {}
        has_primary_key = False
        for field_name, field in cls.__dict__.items():
            if isinstance(field, ForeignKeyReverse) or isinstance(field, ManyToMany):
                field.update_attr(field_name, cls.__tablename__, cls.__db__)
                refed_fields[field_name] = field

            if isinstance(field, Field) or isinstance(field, Model):
                field.name = field_name
                fields[field_name] = field
                if isinstance(field, PrimaryKey):
                    has_primary_key = True
                
        # todo
        if not has_primary_key:
            pk = PrimaryKey()
            pk.name = 'id'
            fields['id'] = pk

        setattr(cls, '__fields__', fields)
        setattr(cls, '__refed_fields__', refed_fields)
        return cls


class Model(metaclass=MetaModel):
    """Base model"""
    __metaclass__ = MetaModel

    def __init__(self, **kwargs):
        for name, field in kwargs.items():
            if name not in self.__fields__.keys():
                raise DatabaseException('Unknown column: {0}, expected {1}.'.format(name, self.__fields__.keys()))
            setattr(self, name, field)

        super(Model, self).__init__()

    @classmethod
    def get(cls, **kwargs):
        return SelectQuery(cls).where(**kwargs).first()

    @classmethod
    def select(cls, *args):
        return SelectQuery(cls, *args)

    @classmethod
    def update(cls, *args, **kwargs):
        return UpdateQuery(cls, *args, **kwargs)

    @classmethod
    def delete(cls, *args, **kwargs):
        return DeleteQuery(cls, *args, **kwargs)

    def save(self):
        base_query = 'insert into {tablename}({columns}) values({items});'
        columns = []
        values = []
        for field_name, field_model in self.__fields__.items():
            if hasattr(self, field_name) and not isinstance(getattr(self, field_name), Field):
                columns.append(field_name)
                values.append(field_model.sql_format(getattr(self, field_name)))

        sql = base_query.format(
            tablename=self.__tablename__,
            columns=', '.join(columns),
            items=', '.join(values)
        )
        cursor = self.__db__.execute(sql=sql, commit=True)
        self.id = cursor.lastrowid

        for name, field in self.__refed_fields__.items():
            if isinstance(field, ForeignKeyReverse) or isinstance(field, ManyToManyBase):
                field.instance_id = self.id

class MongoModel(Model):

    def save(self):
        """columns = []
        values = []
        for field_name, field_model in self.__fields__.items():
            if hasattr(self, field_name) and not isinstance(getattr(self, field_name), Field):
                columns.append(field_name)
                values.append(field_model.sql_format(getattr(self, field_name)))

        sql = base_query.format(
            tablename=self.__tablename__,
            columns=', '.join(columns),
            items=', '.join(values)
        )
        cursor = self.__db__.execute(sql=sql, commit=True)
        self.id = cursor.lastrowid

        for name, field in self.__refed_fields__.items():
            if isinstance(field, ForeignKeyReverse) or isinstance(field, ManyToManyBase):
                field.instance_id = self.id"""
        open("data", "w").write(jsonify(self))


In [69]:
db = Sqlite("dummy.db")

In [70]:
class Question(Model):
    question_text = Char(max_length=200)
    pub_date = DateTime()

    def __repr__(self):
        return str(vars(self))

class Choice(Model):
    question = ManyToMany(Question)
    choice_text = Char(max_length=200)
    votes = Integer()

    def __repr__(self):
        return str(vars(self))

In [71]:
db.create_table(Question)

In [72]:
db.create_table(Choice)

In [73]:
from datetime import datetime 

In [74]:
question = Question(question_text="What is your favorite color?", pub_date=datetime.now())

In [75]:
question.save()

In [76]:
choice_1 = Choice(choice_text="green", votes=0)

In [77]:
choice_1.save()

In [78]:
question.select().all()

[{'question_text': b'What is your favorite color?', 'pub_date': b'2017-05-20 07:02:07', 'id': 1}]

In [79]:
choice_1.question.add(question)

In [83]:
context = {}
for key, val in choice_1.__fields__.items():
    if isinstance(val, ManyToMany):
        rel = getattr(choice_1, key).select().all()
        

{'choice_text': <__main__.Char at 0x7f8a6b3f0828>,
 'id': <__main__.PrimaryKey at 0x7f8a6b476160>,
 'votes': <__main__.Integer at 0x7f8a6b3f0518>}

In [92]:
qs = Choice.select().all()

In [95]:
qs

[{'choice_text': b'green', 'votes': 0, 'id': 1}]

In [101]:
mq.question.relate_table

'question_choice'